In [19]:
import numpy as np
from numpy import linalg as LA
import math
import scipy

In [21]:
# Calculate vector intensity using formula:
# |A| = sqrt(a1^2 + a2^2 + 3^2 +...+ an^2)
# where n = A.length
def intenzitet_vektora(a):
    suma = np.sum(a * a)
    return math.sqrt(suma)

# Find eigenvector for eigenvalue l = 1
# M - matrix
# l - lambda, eigenvalue
# x - eigenvector
# I - identity (unit) matrix
#
# Formula:
# Mx = lx
# (A - lI)x = 0
# |A-lI|x = 0
def svojstven_vekot_i_vrednost(M, l):
    # M must be n*n
    v, e = LA.eig(M)
    for i in range (v.shape[0]):
        if v[i] == 1:
            return e[i]
        
    return []


# oderi sopstveni vektor za labda = 1   
# odredi vektor normalan na njega
# napravi od njega jedinicni vektor
# odredi u' = Au
# odredi ugao fi = arccos(u, u')    
# ako je mesoviti proizvod negativan, 
# promeni p tako da rotacija bude u pozitivnom smeru
def M2AngleAxis(M):
    E = np.eye(3)
    AE = transformation - E
    
    v1 = AE[0, :]
    v2 = AE[1, :]
    v3 = AE[2, :]
    
    p = vector_product(v1,v2)
    
    if isZeroVector(p):
        p = vector_product(v2,v3)
        
        if isZeroVector(p):
            p = vector_product(v1,v3)
            
    euc = math.sqrt(p[0]**2 + p[1]**2 + p[2]**2)
    p = [e/euc for e in p]
    u = np.array([-p[1], p[0], 0])
    euc = math.sqrt((u[0]**2 + u[1]**2 + u[2]**2))
    u = np.array([e/euc for e in u])
    u = u.reshape(3,1)
    u_prim = np.dot(transformation, u)
    u_prim = u_prim.reshape(3,)
    
    u = u.reshape(1,3)
    u = u[0]
    
    euc_uprim = math.sqrt((u_prim[0]**2 + u_prim[1]**2 + u_prim[2]**2))
    angle = np.arccos(np.dot(u,u_prim)/(euc * euc_uprim))
    
    a = np.array([u[0], u[1], u[2],
                  u_prim[0], u_prim[1], u_prim[2],
                  p[0], p[1], p[2]], dtype = 'float').reshape(3,3)

    if np.linalg.det(a) < 0:
        p = (-1) * p
    
    return (p, angle)

def euler2A(fi, teta, ksi):
    cos_fi = math.cos(fi)
    sin_fi = math.sin(fi)
    
    RX = np.array([[1, 0,      0],
                   [0, cos_fi, -sin_fi],
                   [0, sin_fi,  cos_fi]])
    
    cos_teta = math.cos(teta)
    sin_teta = math.sin(teta)
    
    RY = np.array([[cos_teta, 0, sin_teta],
                  [0        , 1, 0],
                  [-sin_teta, 0, cos_teta]])
    
    cos_ksi = math.cos(ksi)
    sin_ksi = math.sin(ksi)
    RZ = np.array([[cos_ksi, -sin_ksi, 0],
                   [sin_ksi, cos_ksi,  0],
                   [0      , 0,        1]])
    
    A = np.dot(RZ,RY)
    A = np.dot(A,RX)
    return A

def a2Euler(A):
    if A[2,0] == -1:
        fi = np.arctan2(-A[0,1], A[1,1])
        teta = np.pi / 2
        ksi = 0
    elif A[2,0] < 1:
        fi = np.arctan2(A[1,0], A[0,0])
        teta = np.arcsin(-A[2,0])
        ksi = np.arctan2(A[2,1], A[2,2])
    elif A[2,0] == 1:
        fi = np.arctan2(A[0,1], -A[1,1])
        teta = -np.pi / 2
        ksi = 0
    else:
        print("uneta losa matrica A")
        return (0,0,0)
    
    return (fi, teta, ksi)

def rodrigez(p, fi):
    fi_sin = np.sin(fi)
    fi_cos = np.cos(fi)
    
    p = p * 1/intenzitet_vektora(p)
    px = np.array([[0,    -p[2], p[1]], 
                   [p[2],  0,    -p[0]], 
                   [-p[1], p[0], 0]])
    
    return np.eye(3) + fi_sin * px + (1 - fi_cos) * px.dot(px)

def axisAngle2q(p, fi):
    # proveri specijalne vrednosti
    #if fi == 0
    #    return (1/3,1/3,1/3,  )
        
    # normalizuj vektor p
    d = math.sqrt(p[0]**2 + p[1]**2 + p[2]**2)
    p = p * 1/d
    
    # pomnozi sa sinusom od fi/2
    p = np.sin(fi/2) * p
    
    # vrati x,y,z i w = cos(fi/2)
    return (p[0], p[1], p[2], np.cos(fi/2));

def q2AxisAngle(q):
    # zelimo ugao izmedju [0,Pi]
    if q[3] < 0:
        q = q*-1
    
    # razdvajamo q na p i d
    a,b,c,d = q
    p = np.array([a,b,c])
        
    if d == 1:
        # ako je |W| = 1 onda je vektor identitet
        p = (1,0,0)
    else:
        # normalizuj vektor
        s = math.sqrt(p[0]**2 + p[1]**2 + p[2]**2)
        p = p * 1/s
    
    # vrati vektor i w = 2arccos(d)
    return (p ,2 * np.arccos(d))

def normalizuj(q):
    s = math.sqrt(q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2)
    return q * 1/s

In [ ]:
# Test template 

# INICIJALIZACIJA

# ojlerovi uglovi
fi = -1 * np.arctan(1/5)
teta = -1 * np.arcsin(10/22)
ksi = np.arctan(16)

# matrica 
A = np.array([[],[],[]])
# kvaternion
q = np.array([])
# vektor
p = np.array([])
# ugao oko vektora
alfa = 

In [ ]:
# Testovi

In [ ]:
# Test euler2A
Ar = euler2A(fi, teta, ksi)
print('Euler2a:')
print(Ar)

In [ ]:
# Test axisAngle
(fir, tetar, ksir) = M2AngleAxis(A)
print('Test axisAngle')
print('old:', fi, teta, ksi)
print('new:', fir,tetar,ksir)

In [ ]:
# Test rodrigez
Ar = rodrigez(p , alfa)
print("rodrigez:")
print(Ar)

In [ ]:
# Test a2Euler
(fir, tetar, ksir) = a2Euler(A)
print("a2Euler")
print('old: ', fi, teta, ksi)
print('new: ', fir, tetar, ksir)

In [ ]:
# Test axisAngle2q
qr = axisAngle2q(p, alfa)
print('axisAngle2q')
print(qr)

In [ ]:
# Test q2AxisAngle
(pr, fir) = q2AxisAngle(q)
print('q2AxisAngle')
print(pr, fir)